<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/endovis18_coco_yolov8_valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#YoloV8:Surgical Instrument Detection
src:https://github.com/Andrewhsin/YOLO-NAS-pytorch

#Download Endovis18 dataset

In [1]:
import gdown

#endovis18 dataset
url = 'https://drive.google.com/uc?id=1lRNAgC-6QgIQd-vum-jr523tYPr6yNM7'
gdown.download(url,'endovis18.zip',quiet=True) 
!unzip -q endovis18.zip 

#Converting endovis18 to COCO format and folder structure<br>
datatset-><br>
&emsp;    images-><br>
&emsp; &emsp; &emsp; train->
&emsp; &emsp;&emsp; &emsp;seq_2_frame000.png, 
 seq_2_frame001.png ... <br>
&emsp; &emsp; &emsp; val->
&emsp; &emsp;&emsp; &emsp;seq_1_frame000.png, 
 seq_1_frame001.png ... <br>
&emsp;    labels-><br>
&emsp; &emsp; &emsp; train->
&emsp; &emsp;&emsp; &emsp;seq_2_frame000.txt, 
 seq_2_frame001.txt ... <br>
&emsp; &emsp; &emsp; val->
&emsp; &emsp;&emsp; &emsp;seq_1_frame000.txt, 
 seq_1_frame001.txt ... <br>

1. Converting .xml of [(x1, y1), (x2, y2)] to .txt (xc, yc, h, w)

In [2]:
import xml.etree.ElementTree as ET
import os
from glob import glob
from tqdm import tqdm

root_dir = "endovis18/"
dest_dir = root_dir

class_name_to_id_mapping = {
    "kidney": 0,
    "bipolar_forceps": 1,
    "prograsp_forceps": 2,
    "large_needle_driver": 3,
    "monopolar_curved_scissors": 4,
    "ultrasound_probe": 5,
    "suction": 6,
    "clip_applier": 7,
    "stapler": 8,
}


# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()

    # Initialise the info dict
    info_dict = {}
    info_dict["bboxes"] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name
        if elem.tag == "filename":
            info_dict["filename"] = elem.text

        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))

            info_dict["image_size"] = tuple(image_size)

        # Get details of the bounding box
        elif elem.tag == "objects":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text

                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)
            info_dict["bboxes"].append(bbox)

    info_dict["image_size"] = tuple([1280, 1024, 3])

    return info_dict


# print(extract_info_from_xml('dataset/instruments18/seq_1/xml/frame000.xml'))


def convert_to_yolov5(info_dict, ann):
    print_buffer = []

    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())

        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width = b["xmax"] - b["xmin"]
        b_height = b["ymax"] - b["ymin"]

        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]
        b_center_x /= image_w
        b_center_y /= image_h
        b_width /= image_w
        b_height /= image_h

        # Write the bbox details to the file
        print_buffer.append(
            "{} {:.3f} {:.3f} {:.3f} {:.3f}".format(
                class_id, b_center_x, b_center_y, b_width, b_height
            )
        )

    # Name of the file which we have to save
    save_file_name = os.path.splitext(ann)[0] + ".txt"

    # Save the annotation to disk
    print("\n".join(print_buffer), file=open(save_file_name, "w"))


# Get the annotations
annotations = glob(root_dir + "*/xml/*.xml")

# # Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict, ann)

annotations = glob(root_dir + "*/xml/*.txt")
print('done!')

100%|██████████| 2007/2007 [00:00<00:00, 3809.44it/s]

done!


2. Rearranging folders as COCO dataset

In [3]:
from glob import glob
import shutil
import os
from PIL import Image

root_dir = "endovis18/"
path = glob(root_dir + "*/xml/*.txt")
endovis_coco_img_path_train = 'endovis18_coco/images/train'
endovis_coco_img_path_val = 'endovis18_coco/images/val'
endovis_coco_label_path_train = 'endovis18_coco/labels/train'
endovis_coco_label_path_val = 'endovis18_coco/labels/val'

os.makedirs(endovis_coco_img_path_train, exist_ok=True)
os.makedirs(endovis_coco_img_path_val, exist_ok=True)
os.makedirs(endovis_coco_label_path_train, exist_ok=True)
os.makedirs(endovis_coco_label_path_val, exist_ok=True)

#validation set with the seq 1, 5, 16 following: https://ieeexplore.ieee.org/abstract/document/9944843
val_seq = [1, 5, 16]
path_all = []
for seq in val_seq:
    path_all.extend(glob(root_dir + "seq_{}/xml/*.txt".format(seq)))

for path in path_all:
    endovis_coco_label_path_val_full = os.path.join(endovis_coco_label_path_val, path.split('/')[1] + '_' + os.path.basename(path))
    shutil.copyfile(path, endovis_coco_label_path_val_full)
    img_path = os.path.join(path.split('/')[0], path.split('/')[1],'left_frames', os.path.basename(path[:-3])+'png')
    endovis_coco_img_path_val_full = os.path.join(endovis_coco_img_path_val, img_path.split('/')[1] + '_' + os.path.basename(img_path))
    shutil.copyfile(img_path, endovis_coco_img_path_val_full)
    

# Training set with the remaining seq following: https://ieeexplore.ieee.org/abstract/document/9944843
train_seq = [2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15]
path_all = []
for seq in train_seq:
    path_all.extend(glob(root_dir + "seq_{}/xml/*.txt".format(seq)))

for path in path_all:
    endovis_coco_label_path_train_full = os.path.join(endovis_coco_label_path_train, path.split('/')[1] + '_' + os.path.basename(path))
    shutil.copyfile(path, endovis_coco_label_path_train_full)
    img_path = os.path.join(path.split('/')[0], path.split('/')[1],'left_frames', os.path.basename(path[:-3])+'png')
    endovis_coco_img_path_train_full = os.path.join(endovis_coco_img_path_train, img_path.split('/')[1] + '_' + os.path.basename(img_path))
    shutil.copyfile(img_path, endovis_coco_img_path_train_full)

#Download Code and Trained Weights

installation "super_gradients" lib


In [17]:
!pip -q install super-gradients

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 65.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py)

Unfortunately, need to edit super_gradients/training/utils/utils.py.<br>
To edit the file please pressed cmd (in mac) or ctrl in windows + click on this file dir

In [ ]:
#1. To edit the file please pressed cmd (in mac) or ctrl in windows + click on this file dir. To get python version you need to run the code or check the installed directory for super_gradients
/usr/local/lib/python3.10/dist-packages/super_gradients/training/utils/utils.py

# Go to line 595:
#current: if isinstance(inputs, collections.Iterable) and not isinstance(inputs, str):
#modify to: if isinstance(inputs, collections.abc.Iterable) and not isinstance(inputs, str):

Download the code:

In [4]:
!git clone https://github.com/Andrewhsin/YOLO-NAS-pytorch
%cd YOLO-NAS-pytorch

Cloning into 'YOLO-NAS-pytorch'...
remote: Enumerating objects: 1636, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 1636 (delta 16), reused 0 (delta 0), pack-reused 1559
Receiving objects: 100% (1636/1636), 80.76 MiB | 15.29 MiB/s, done.
Resolving deltas: 100% (367/367), done.
Updating files: 100% (1350/1350), done.
/content/YOLO-NAS-pytorch


#Training
Validation on trained weights following: https://ieeexplore.ieee.org/abstract/document/9944843

1. Creating yml file to set all dataset dirs:

In [6]:
import yaml

with open("dataset/data.yaml") as f:
     list_doc = yaml.safe_load(f)

list_doc['Dir'] = '/content/endovis18_coco/'
list_doc['images']['train'] = 'images/train'
list_doc['images']['val'] = 'images/val'
list_doc['images']['test'] = 'images/val'

list_doc['labels']['train'] = 'labels/train'
list_doc['labels']['val'] = 'labels/val'
list_doc['labels']['test'] = 'labels/val'

with open("dataset/data.yaml", "w") as f:
    yaml.dump(list_doc, f)

2. Training

In [2]:
!python3 train.py --data dataset/data.yaml --batch 6 --epoch 100 --model yolo_nas_m

The console stream is logged into /root/sg_logs/console.log
[2023-06-10 14:59:22] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023-06-10 14:59:28.526731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 14:59:32.771467: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-06-10 14:59:36] INFO - utils.py - NumExpr defaulting to 2 threads.
[2023-06-10 14:59:37] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-06-10 14:59:37] WA